In [2]:
!HF_ENDPOINT=https://hf-mirror.com 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import PeftModel, PeftConfig
import torch

In [3]:
# Loading PEFT model
PEFT_MODEL = "idegroup/PhyAssist"
config = PeftConfig.from_pretrained(PEFT_MODEL)

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=False,
)

# model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)
# model = model.merge_and_unload()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
import gradio as gr
from transformers import pipeline, TextIteratorStreamer
from threading import Thread


def format_prompt(message, history):
    prompt = "<s>"
    # prompt += f"[INST] As a psychology counselor assistant, provide an assessment and plan for the following counseling notes. Please present a summary, don't make it so long. Present in lines.: {message} [/INST]"
    # for user_prompt, bot_response in history:
    #     prompt += f"[INST] {user_prompt} [/INST]"
    #     prompt += f" {bot_response}</s> "
    prompt += f"[INST]As a psychology counselor assistant, you need to provide an assessment and plan for the following counseling post and question. \nYour answer shoule be formed by to parts: the problem assessment and problem solution plan. Don't make your generation so long.  Please present the plan in lines. \nPost:{message} Question: Does the patient has mental health problem? [/INST]"
    return prompt
def generate(
    prompt, history, temperature=0.9, max_new_tokens=1024, top_p=0.95, repetition_penalty=1.0,
):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    formatted_prompt = format_prompt(f"{prompt}", history)
    # print(formatted_prompt)
    # stream = client(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to('cuda:0')
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=320)
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    generated_text = ""
    for new_text in streamer:
        generated_text += new_text
        yield generated_text
    thread.join()
    return generated_text


examples=[
    ["Patient is feeling stressed due to work and has trouble sleeping.", None, None, None, None, None],
    ["Client is dealing with relationship issues and is seeking advice on communication strategies.", None, None, None, None, None],
    ["Individual has recently experienced a loss and is having difficulty coping with grief.", None, None, None, None, None],
]

gr.ChatInterface(
    fn=generate,
    chatbot=gr.Chatbot(show_label=False, show_share_button=False, show_copy_button=True, likeable=True, layout="panel"),
    title="Psychological Assistant: Expert in Assessment and Strategic Planning",
    description="Enter counseling notes to generate an assessment and plan.",
    examples=examples,
    concurrency_limit=20,
).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://39a39b2524ee9c0f57.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
from transformers import MistralForCausalLM